# Viewing Trending Videos Database

In this notebook we take a look at the database we created, which contains several metrics on trending videos of YouTube.

The database is automatically updated every day by inserting 50 new rows, the maximum number of videos provided in the YouTube API trending videos list. We use Amazon Lambda with Amazon EventBridge Scheduler to invoke a python function every day. This function utilizes YouTube API to get the trending videos list and retrieve several metrics of the videos in the list. The data is written in a PostgreSQL database in Amazon RDS.

The python script utilized in the Lambda function to collect data is contained as a separate file in this folder.

The goal of this project is to eventually analyze trending videos after collecting data for a few months.

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from configparser import ConfigParser

In [2]:
# Load configuration from file
config = ConfigParser()
config.read('config.ini')

# PostgreSQL database configuration
host = config.get('database', 'host')
user = config.get('database', 'user')
password = config.get('database', 'password')
database = config.get('database', 'database')
port = 5432

In [3]:
# Construct the database URL
db_url = f"postgresql://{user}:{password}@{host}:{port}/{database}"

# Create a SQLAlchemy engine
engine = create_engine(db_url)

query = "SELECT * FROM youtube_metrics"
df = pd.read_sql_query(query, engine)

df

,video_id,trending_date,publish_date,title,duration,views,likes,comments
0,Wjij-OX9RKI,2023-12-03,2023-12-02,Fortnite Battle Royale Chapter 5 Launch Trailer,0 days 00:01:30,4874502,230120,16151
1,F9U-yoJbgWs,2023-12-03,2023-12-02,The Boys – Season 4 Official Teaser Trailer | ...,0 days 00:01:50,5944265,314273,11029
2,0kQ8i2FpRDk,2023-12-03,2023-12-02,Fallout - Teaser Trailer | Prime Video,0 days 00:02:34,4738672,140347,12927
3,HQ8H5gqGA34,2023-12-03,2023-12-02,House of the Dragon Season 2 | Official Teaser...,0 days 00:01:26,3250473,130514,4834
4,okvG6H0UaIs,2023-12-03,2023-12-02,Top 10 GIFS of ALL TIME,0 days 00:26:24,2063101,83318,3086
...,...,...,...,...,...,...,...,...
495,GmshXm-Hj-M,2023-12-13,2023-12-10,"Best MrBeast Chocolate Art Wins $10,000!",0 days 00:24:56,654397,21229,1307
496,vhuouufaypY,2023-12-13,2023-12-11,Who Can Survive the Longest in Mario Wonder?,0 days 00:26:03,313604,8156,443
497,IC9a4Dkbq-g,2023-12-13,2023-12-10,The Last of Them,0 days 00:07:04,2129148,103172,3285
498,NJNSgOE0AvE,2023-12-13,2023-12-10,PACERS vs LAKERS | NBA IN-SEASON TOURNAMENT CH...,0 days 00:10:01,2002168,21228,3066


In [4]:
query = """SELECT trending_date td, COUNT(*) entry_count,
        (SELECT COUNT(*) FROM youtube_metrics) total
        FROM youtube_metrics
        GROUP BY td
        ORDER BY td
        """
pd.read_sql_query(query, engine)

,td,entry_count,total
0,2023-12-03,50,500
1,2023-12-04,50,500
2,2023-12-05,50,500
3,2023-12-06,50,500
4,2023-12-07,50,500
5,2023-12-09,50,500
6,2023-12-10,50,500
7,2023-12-11,50,500
8,2023-12-12,50,500
9,2023-12-13,50,500


This dataframe contains the current status of our database after ten days of inserting new data.

Note that both `video_id` and `trending_date` are primary keys, so if a video appears in the trending videos list for more than a day, that is also recorded.